In [10]:
!pip install import_ipynb
import pandas as pd
import matplotlib.pyplot as plt
import import_ipynb
import preprocess_with_val
import numpy as np
np.random.seed(530)
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve,auc,confusion_matrix
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr, spearmanr, ks_2samp
from sklearn.feature_selection import chi2
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import StratifiedKFold
import collections
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import random

In [11]:
race_list = ['patient_race_american_indian_or_alaska_native',
 'patient_race_asian',
 'patient_race_hispanic',
 'patient_race_native_hawaiian_or_other_pacific_islander',
 'patient_race_other',
 'patient_race_white',
 'patient_race_african_american_or_black']

# import pickle
# f = open(r'columns_60', 'rb')
# fin_cols = pickle.load(f)

#load dataset
cov_train = pd.read_csv("preprocessed_covariate_race_matrix_post_surgery_7_days_train_{}.csv".format("1225"))
cov_test = pd.read_csv("preprocessed_covariate_race_matrix_post_surgery_7_days_test_{}.csv".format("1225"))
cov_val = pd.read_csv("preprocessed_covariate_race_matrix_post_surgery_7_days_val_{}.csv".format("1225"))

cov_train_final, cov_test_final, cov_val_final = preprocess_with_val.preprocess(cov_train, cov_test, cov_val)
cov_train_final.reset_index(drop = True, inplace = True)
cov_test_final.reset_index(drop = True, inplace =True)
cov_val_final.reset_index(drop = True, inplace =True)

cov_train_final.drop(race_list + ['patient_gender_male'], axis = 1, inplace = True)
cov_test_final.drop(race_list + ['patient_gender_male'], axis = 1, inplace = True)
cov_val_final.drop(race_list + ['patient_gender_male'], axis = 1, inplace = True)

In [12]:
dtype = torch.FloatTensor
#model class
class logisticregression(torch.nn.Module):
  def __init__(self, input_dim, output_dim, seed = 530):
    super(logisticregression, self).__init__()
    # self.layer = nn.Sequential(nn.Linear(input_dim, output_dim),
    #                            nn.Dropout(p = 0.8),
    #                            nn.BatchNorm1d(128),
    #                            nn.ReLU(),


                               
    self.seed = seed

    self.linear = torch.nn.Linear(input_dim, 64)
    self.linear2 = torch.nn.Linear(64, 1)
    # self.linear3 = torch.nn.Linear(128, 64)
    # self.linear4 = torch.nn.Linear(64, 32)
    # self.linear5 = torch.nn.Linear(32, output_dim)
    self.bn = nn.BatchNorm1d(input_dim)
    self.bn1 = nn.BatchNorm1d(64)
    self.sigmoid = torch.nn.Sigmoid()
    self.relu = torch.nn.ReLU()
    self.drop1 = torch.nn.Dropout(p = 0.8)

    
    self.linear.apply(self.init_weights)
    self.linear2.apply(self.init_weights)
    # self.linear3.apply(self.init_weights)
    # self.linear4.apply(self.init_weights)
    # self.linear5.apply(self.init_weights)


  def init_weights(self, m):
    if type(m) == nn.Linear:
      #m.weight.data.fill_(0.05)
      torch.manual_seed(self.seed)
      # nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
      # nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out', nonlinearity='relu')
      torch.nn.init.orthogonal_(m.weight, gain=nn.init.calculate_gain('relu'))
      # torch.nn.init.sparse_(m.weight, sparsity = 0.1, std = 0.01)
      m.bias.data.fill_(0.0)

  def forward(self, x):
    # output1 = self.linear(x)
    # x = self.bn(x)
    output1 = self.relu(self.bn1(self.drop1(self.linear(x))))
    # output1 = self.relu(self.drop1(self.linear(x)))
    # output1 = self.bn(output1)
    # drop1 = torch.nn.Dropout(p = 0.5)(output1)
    # output1 = self.relu(drop1)
    

    output2 = self.linear2(output1)
    # output2 = self.relu(self.bn2(self.drop2(self.linear(output1))))
    # output2 = self.relu(output2)
    # drop2 = torch.nn.Dropout(p = 0.5)(output2)

    # output3 = self.linear3(output2)
    # output3 = self.relu(output3)
    # drop3 = torch.nn.Dropout(p = 0.3)(output3)

    # output4 = self.linear4(drop3)
    # output4 = self.relu(output4)
    # drop4 = torch.nn.Dropout(p = 0.3)(output4)

    # output5 = self.linear5(drop4)
    output5 = self.sigmoid(output2)
    return output5


In [13]:
loss_func = torch.nn.BCELoss()
# cov_train_final, _ = preprocess.preprocess(cov_train, cov_test)
train_data = cov_train_final
def scale(x_scale, train_scale = cov_train_final):
  from sklearn.preprocessing import MinMaxScaler
  # train_scale = train_data.sample(frac = 1, random_state = 530).reset_index(drop = True)
  # train_scale = train_data
  scaler = MinMaxScaler()
  # scaler = StandardScaler()
  X = train_scale.drop(['ssi'], axis = 1)
  X = np.array(X)
  scaler.fit(X)
  res = scaler.transform(x_scale)
  res = pd.DataFrame(res, columns = train_scale.drop(['ssi'], axis = 1).columns)
  return res

In [14]:
def train_dataloader(batch = 1000, train_data = cov_train_final):
  # print(batch)
  train = train_data.sample(frac = 1, random_state = 530).reset_index(drop = True)
  X_train = train.drop(['ssi'], axis = 1)
  X_train = scale(X_train.values)
  X_train = X_train.values
  y_train = train['ssi'].values

  # batch = 1000
  if batch == "all":
      yield X_train, y_train
  else:
    idx = 0
    while idx < len(X_train):
      if idx + batch < len(X_train):
        yield X_train[idx:idx + batch, :], y_train[idx:idx + batch]
      else:
        yield X_train[idx:len(X_train), :], y_train[idx:len(X_train)]
      idx = idx + batch

In [15]:
def conf_mat(test, pred):
  conf_matrix = confusion_matrix(test, pred)
  TP = conf_matrix[1][1]
  TN = conf_matrix[0][0]
  FP = conf_matrix[0][1]
  FN = conf_matrix[1][0]
  return TP, TN, FP, FN

In [16]:
def weighted_cross_entropy(y_pred, y_true):
  logloss = (1-y_true)*torch.log(1e-6 + 1 - y_pred)
  logloss += y_true*torch.log(1e-6 + y_pred)
  return -torch.mean(logloss)

In [17]:
def predict(p_model, x, logit = False, thresh = 0.5):
  p_model.eval()
  x_ = x.drop(['ssi'], axis = 1).values
  y_ = x['ssi'].values
  x_= scale(x_)
  x_ = x_.values
  x_ = Variable(torch.from_numpy(x_).type(dtype))
  y_ = Variable(torch.from_numpy(y_).type(dtype))
  y_ = y_.view(x_.size()[0], )
  output_g = p_model(x_)
  y_pred = output_g
  # y_pred = output_g.detach().numpy()
  if logit:
    y_pred[y_pred >= thresh] = 1
    y_pred[y_pred < thresh] = 0
  return y_pred, y_

In [ ]:
#baseline performance
from torch import autograd
output_dim = 1
epochs = 100
lr_rate = 1e-2

#input_dim = len(fin_cols) - 1
input_dim = 264
mod = logisticregression(input_dim = input_dim, output_dim = output_dim)

#for i, r in enumerate(race_list):
optimizer = torch.optim.Adam(mod.parameters(), lr = 1e-4)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr = 1e-4, max_lr = 5e-4, cycle_momentum = False)
# lambda1 = lambda epoch: 0.65 ** epoch
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 10, eta_min=1e-4)   #issa good
# scheduler =  torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=1e-2,step_size_up=5,mode="exp_range", cycle_momentum = False)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=35, epochs=100)
min_val_loss = float('inf')
batch_loss = []
#training loop
opt_array = []
loss_array = []
for epoch in range(epochs):
    print("epoch: ", epoch)
    
    b = 0
    # if (epoch+1)%10 == 0:
    #   optimizer.param_groups[0]['lr'] /= 10
    # for feat, label in train_dataloader():
    for feat, label in train_dataloader(batch = 256):
    # for feat, label in batch_data.values():
      mod.train()
      # feat, label = batch_data[i]
      b += 1
      x = Variable(torch.from_numpy(feat).type(dtype))
      y = Variable(torch.from_numpy(label).type(dtype))
      #with autograd.detect_anomaly():
      optimizer.zero_grad()
      outputs = mod(x)
      y = torch.reshape(y, outputs.shape)
      loss = loss_func(outputs, y) 
      loss_array.append(loss)
      loss.backward()
      optimizer.step()
      scheduler.step()
    opt_array.append(optimizer.param_groups[0]['lr'])
    # print(b)
    print("loss: ", sum(loss_array).detach().numpy()/len(loss_array))
    batch_loss.append(sum(loss_array).detach().numpy()/len(loss_array))
    y_pred_val, y_val = predict(mod, cov_val_final)
    y_val = torch.reshape(y_val, y_pred_val.shape)
    val_loss = loss_func(y_pred_val, y_val)
    print("val_loss: ", val_loss.detach().numpy())
    if val_loss < min_val_loss:
      min_val_loss = val_loss
      torch.save(mod, 'best_model')
      print("saving model due to new min val loss")
      best_epoch = epoch
    print("best_epoch: ", best_epoch)

In [19]:
best_mod = torch.load('best_model')
#test data
# best_mod.eval()
y_pred, y_test = predict(best_mod, cov_test_final)
y_pred = y_pred.view(-1)
print(loss_func(y_pred, y_test))
y_pred = y_pred.detach().numpy()
y_test = y_test.detach().numpy()

#train data
y_pred_train, y_train = predict(best_mod, cov_train_final)
y_pred_train = y_pred_train.detach().numpy()
y_train = y_train.detach().numpy()

pr, re, thresh_test = precision_recall_curve(y_test, y_pred)
print("AUPRC test: ", auc(re, pr))

pr, re, thresh_test = precision_recall_curve(y_train, y_pred_train)
print("AUPRC train: ", auc(re, pr))

fpr_test, tpr_test, _ = roc_curve(y_test, y_pred)
print("ROC test: ", auc(fpr_test, tpr_test))

fpr_train, tpr_train, _ = roc_curve(y_train, y_pred_train)
print("ROC train: ", auc(fpr_train, tpr_train))

y_pred[y_pred >= 0.4] = 1
y_pred[y_pred < 0.4] = 0
TP, TN, FP, FN = conf_mat(y_test, y_pred)
rec = (TP / float(TP + FN))
prec = (TP / float(TP + FP))
print("F1 score: ", 2*rec*prec/(rec+prec))

tensor(0.2065, grad_fn=<BinaryCrossEntropyBackward>)
AUPRC test:  0.20478487094276046
AUPRC train:  0.3135030932836749
ROC test:  0.8541254636944233
ROC train:  0.8922915557325474
F1 score:  0.2901960784313726
